In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import time

import load_data

/home/molly/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


ModuleNotFoundError: No module named 'load_data'

In [2]:
tcga = load_data.read_data_sets("./data/mRNA_lognorm_MinMaxScaled.csv")

### Python batching

In [34]:
tcga.train.reset_epoch()

t0 = time.time()

epochs_completed = 0
while tcga.train.epochs_completed <= 10:
    batch_x, batch_y = tcga.train.next_batch(100)
    y1 = batch_y["tissue"]
    y2 = batch_y["gender"]
    y3 = batch_y["tumor"]
    if tcga.train.epochs_completed > epochs_completed:
        epochs_completed += 1
        print("epoch completed", tcga.train.epochs_completed) 
            
print("time elapsed:", time.time() - t0, "seconds")

epoch completed 1
epoch completed 2
epoch completed 3
epoch completed 4
epoch completed 5
epoch completed 6
epoch completed 7
epoch completed 8
epoch completed 9
epoch completed 10
epoch completed 11
time elapsed: 4.052863359451294 seconds


### Tensorflow Dataset

In [1]:
features_placeholder = tf.placeholder(tf.float32, tcga.train.X.shape)
label_tissue_placeholder = tf.placeholder(tf.float32, tcga.train.y["tissue"].shape)
label_gender_placeholder = tf.placeholder(tf.float32, tcga.train.y["gender"].shape)
label_tumor_placeholder = tf.placeholder(tf.float32, tcga.train.y["tumor"].shape)

dataset = tf.data.Dataset.from_tensor_slices((features_placeholder, 
                                              label_tissue_placeholder,
                                              label_gender_placeholder, 
                                              label_tumor_placeholder))
dataset=dataset.repeat(10)
dataset=dataset.batch(1000)

dataset = dataset.prefetch(10)

iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

sess = tf.Session()


sess.run(iterator.initializer, feed_dict={features_placeholder: tcga.train.X,
                                          label_tissue_placeholder: tcga.train.y["tissue"],
                                          label_gender_placeholder: tcga.train.y["gender"],
                                          label_tumor_placeholder: tcga.train.y["tumor"],})

NameError: name 'tf' is not defined

In [54]:
t0 = time.time()
for i in range(1000000):
    try:
        x, y1, y2, y3 = sess.run(next_element)
    except tf.errors.OutOfRangeError:
        break
        
print("time elapsed:", time.time() - t0, "seconds")

time elapsed: 3.224940299987793 seconds
